In [1]:
from h5rdmtoolbox.h5wrapper import H5PIV
from h5rdmtoolbox import tutorial

import os

c:\users\da4323\documents\programming\github\h5rdmtoolbox\h5rdmtoolbox\h5wrapper\h5piv.py:675: SpecialDatasetRegistrationWarning: registration of accessor <class 'h5rdmtoolbox.h5wrapper.h5piv.PIVDisplacementDataset'> under name 'DisplacementVector' for type <class 'h5rdmtoolbox.h5wrapper.h5piv.H5PIVGroup'> is overriding a preexisting attribute with the same name.
  class PIVDisplacementDataset(DisplacementDataset):


In [2]:
with tutorial.get_H5PIV('minimal_flow', mode='r') as h5:
    h5.dump()

# Special datasets: `VelocityVector`

Velocity is a vector quantity. In the HDF file each component generally is stored as an individual data variable. To get the full vector in one variable the attribute `VelocityVector` can be called. Slicing this object will assigned it with the respectve arrays from the HDF datasets. In this case x- and y-velociy datasets are siced and merged into a `xr.Dataset`. The `VelocityVector` class is wrapped around the `xr.Dataset` class and has additional methods, like `compute_magnitude()`.

Note, that `VelocityVector` is not a property of `H5PIV` by default but is added afterwards, similar to how dataarray-ccessors are added to an `xr.DataArray`.

In [3]:
with tutorial.get_H5PIV('minimal_flow', mode='r') as h5:
    h5.dump()
    vel = h5.VelocityVector[:]

`VeloctyVector` has most properties, that `xr.Dataset has`, too:

In [4]:
vel

SpecialDataset of group "/" (closed)
<xarray.Dataset>
Dimensions:  (z: 2, time: 5, y: 110, x: 127)
Coordinates:
  * z        (z) float32 -5.0 0.0
  * time     (time) float32 0.0 0.2 0.4 0.6 0.8
  * y        (y) float64 -15.22 -14.33 -13.44 -12.56 ... 79.0 79.89 80.78 81.67
  * x        (x) float64 -22.11 -21.22 -20.33 -19.44 ... 87.22 88.11 89.0 89.89
Data variables:
    u        (z, time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    v        (z, time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    units:      m/s
    long_name:  vector data

## Post-processing with `H5PIV`

Based on standard names velocity, velocity gradients and other variables are identified. Each group therefore should have only one velocity vector, otherwise it is not clear from which to compute e.g. the turbulent kinetic energy.<br>
From each group, multiple fluid-specific post-processing methods can be called. They store the result in the respective group if required data was correcly identified:

In [5]:
with tutorial.get_H5PIV('vortex_snapshot', mode='r+') as h5:
    h5.dump()